In [ ]:
# Instalar las bibliotecas necesarias
!pip install Flask psycopg2 google-cloud-language googlemaps

from flask import Flask, render_template, request, jsonify
from googlemaps import Client as GoogleMaps
import psycopg2
import os

app = Flask(__name__)

# Configuración de la base de datos (PostgreSQL)
# En Colab, puedes usar variables de entorno o un archivo de configuración
# para almacenar tus credenciales de la base de datos.
# Ejemplo usando variables de entorno:
app.config['DB_HOST'] = os.environ.get('DB_HOST')
app.config['DB_USER'] = os.environ.get('DB_USER')
app.config['DB_PASSWORD'] = os.environ.get('DB_PASSWORD')
app.config['DB_NAME'] = os.environ.get('DB_NAME')

# Configuración de la API de Google Maps
app.config['GOOGLE_MAPS_API_KEY'] = os.environ.get('GOOGLE_MAPS_API_KEY')


def get_db_connection():
    """
    Función para establecer la conexión con la base de datos.
    """
    try:
        conn = psycopg2.connect(
            host=app.config['DB_HOST'],
            user=app.config['DB_USER'],
            password=app.config['DB_PASSWORD'],
            dbname=app.config['DB_NAME']
        )
        return conn
    except psycopg2.Error as e:
        print(f"Error al conectar a la base de datos: {e}")
        return None  # Importante manejar el caso de error


@app.route('/seccion1', methods=['GET', 'POST'])
def seccion1():
    """
    Procesa los datos demográficos y socioeconómicos del formulario.
    """
    if request.method == 'POST':
        pais = request.form['pais']
        departamento = request.form['departamento']
        municipio = request.form['municipio']
        edad = int(request.form['edad'])
        genero = request.form['genero']
        nivel_educativo = request.form['nivel_educativo']
        situacion_laboral = request.form.getlist('situacion_laboral')
        acceso_tecnologia = request.form.getlist('acceso_tecnologia')

        # 1. Validación geográfica con Google Maps API
        gmaps = GoogleMaps(app.config['GOOGLE_MAPS_API_KEY'])
        try:
            results = gmaps.geocode(municipio + ', ' + departamento + ', ' + pais)
            if results:
                latitude = results[0]['geometry']['location']['lat']
                longitude = results[0]['geometry']['location']['lng']
            else:
                return jsonify({'error': 'Municipio no encontrado'}), 400
        except Exception as e:
            return jsonify({'error': f'Error en Google Maps API: {e}'}), 500

        # 2. Validación de edad
        if edad < 15 or edad > 90:
            return jsonify({'error': 'Edad fuera del rango permitido'}), 400

        # 3. Guardar los datos en la base de datos
        conn = get_db_connection()
        if conn:  # Verificar si la conexión fue exitosa
            try:
                cur = conn.cursor()
                cur.execute(
                    """
                    INSERT INTO datos_demograficos (
                        pais, departamento, municipio, edad, genero,
                        nivel_educativo, situacion_laboral, acceso_tecnologia,
                        latitud, longitud
                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    """,
                    (pais, departamento, municipio, edad, genero,
                     nivel_educativo, ', '.join(situacion_laboral), ', '.join(acceso_tecnologia),
                     latitude, longitude)
                )
                conn.commit()
                cur.close()
                conn.close()
                return jsonify({'message': 'Datos guardados correctamente'}), 200
            except psycopg2.Error as e:
                conn.rollback()  # Revertir cualquier cambio en caso de error
                return jsonify({'error': f'Error al guardar en la base de datos: {e}'}), 500
            finally:
                if conn:  # Asegurar que la conexión se cierre siempre
                    conn.close()
        else:
            return jsonify({'error': 'No se pudo conectar a la base de datos'}), 500

    return render_template('seccion1.html')  # Un template HTML para el formulario


if __name__ == '__main__':
    app.run(debug=True)

  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40715 sha256=2115d001f6ab098e95879f5790b2656eb5415ed97e7b7eab7f0b60c864c60106
  Stored in directory: /root/.cache/pip/wheels/f1/09/77/3cc2f5659cbc62341b30f806aca2b25e6a26c351daa5b1f49a
Successfully built googlemaps
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
